<a href="https://colab.research.google.com/github/gergogomori/brain_wide_pes/blob/main/pes_firing_rates.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install elephant


In [ ]:
# RESTART SESSION

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pickle

import quantities as pq
import neo
from elephant.statistics   import instantaneous_rate
from elephant.kernels      import GaussianKernel

rng = np.random.default_rng()


# Parameters

In [ ]:
t_lims = (-350.0, 350.0) # ms

min_avg_fr = 1.0 # Hz

sampl_rate = 10.0  # ms
ker_sig    = 20.0  # ms

t_dec_range = (0, 250) # ms

curr_kernel = GaussianKernel(sigma=ker_sig * pq.ms)
time_axis   = np.arange(t_lims[0] + sampl_rate / 2, t_lims[1] - sampl_rate / 2 + 1, sampl_rate)

print(f'Time axis: {time_axis} ms')


# Loading the data

In [ ]:
with open('*** Update the folder path here to match your directory structure ***', 'rb') as f:
    spikes_all = pickle.load(f)


# Discarding neurons below the minimum firing rate

In [ ]:
eids_contrs = sorted(list(spikes_all.keys()))

print(f'Starting number of sessions: {len(eids_contrs)}')

for curr_eid_contr in eids_contrs:

    silent_neurons = []
    curr_neurons   = sorted(list(spikes_all[curr_eid_contr].keys()))

    for neuron in curr_neurons:

        n_spikes = 0

        for trial in spikes_all[curr_eid_contr][neuron].keys():

            curr_spiketrain = spikes_all[curr_eid_contr][neuron][trial]

            n_spikes += len(curr_spiketrain[np.logical_and(curr_spiketrain > t_dec_range[0], curr_spiketrain < t_dec_range[1])])

        if (n_spikes / (len(spikes_all[curr_eid_contr][neuron].keys()) * ((t_dec_range[1] - t_dec_range[0]) / 1000))) < min_avg_fr:
            silent_neurons.append(neuron)

    for neuron in curr_neurons:
        if neuron in silent_neurons:
            spikes_all[curr_eid_contr].pop(neuron)

for curr_eid_contr in eids_contrs:
    if len(spikes_all[curr_eid_contr].keys()) == 0:
        spikes_all.pop(curr_eid_contr)

print(f'Final number of sessions: {len(list(spikes_all.keys()))}')


# Converting spiketrains to firing rates

In [ ]:
fr_all = {}
eids_contrs = sorted(list(spikes_all.keys()))

for ind_eid_contr, curr_eid_contr in enumerate(eids_contrs):

    print(f'Done with {np.round(ind_eid_contr / len(eids_contrs) * 100, 2)} % of the data.')

    fr_all[curr_eid_contr] = {}

    print(len(list(spikes_all[curr_eid_contr].keys())))

    for neuron in spikes_all[curr_eid_contr].keys():

        fr_all[curr_eid_contr][neuron] = {}

        for trial in spikes_all[curr_eid_contr][neuron].keys():

            curr_spikes = spikes_all[curr_eid_contr][neuron][trial]
            curr_spikes = curr_spikes[np.logical_and(curr_spikes > t_lims[0], curr_spikes < t_lims[1])]
            curr_sp_neo = neo.SpikeTrain(curr_spikes, t_start=t_lims[0] * pq.ms, t_stop=t_lims[1] * pq.ms, units=pq.ms)

            fr_all[curr_eid_contr][neuron][trial] = instantaneous_rate(curr_sp_neo, sampling_period=sampl_rate * pq.ms, kernel=curr_kernel).magnitude.flatten()

            assert(len(fr_all[curr_eid_contr][neuron][trial]) == len(time_axis))

with open('*** Update the folder path here to match your directory structure ***', 'wb') as f:
    pickle.dump(fr_all, f, pickle.HIGHEST_PROTOCOL)
